# Fibre packer demo

*Author: Vedrana Andersen Dahl (vand@dtu.dk)*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vedranaa/fibre-pack/blob/main/fibre_packer_demo.ipynb)



In [ ]:
import os
if not os.path.isfile('fibre_packer.py'):
    !wget 'https://raw.githubusercontent.com/vedranaa/fibre-pack/main/fibre_packer.py' -q
import fibre_packer as fp

## Setting up and optimizing the first and last slice

In [ ]:
domain_radius = 70  # Domain radius
fibre_radius_mean = 2  # Mean fibre radius
fibre_radius_sigma = 0.1 * fibre_radius_mean  # Standard deviation of fibre radius
fibre_volume_fraction = 65  # Desired fibre volume fraction
number_slices = 20 # Number of slices to generate
misalignment = 'high'

In [ ]:
fib = fp.from_fvf(domain_radius, fibre_volume_fraction, fibre_radius_mean, fibre_radius_sigma)
fib.show_radii_distribution()

In [ ]:
fib.initialize_start_slice()
fib.show_slice('start','Initial start slice')

In [ ]:
losses = fib.optimize_slice_heuristic('start', iters=100, repetitions=10)

In [ ]:
fib.show_losses(losses)
fib.show_slice('start','Optimized start slice', show_issues=True)

In [ ]:
fib.initialize_end_slice(misalignment)
fib.show_slice('end','Initial end slice')

In [ ]:
losses = fib.optimize_slice_heuristic('end', iters=100, repetitions=10)

In [ ]:
fib.show_losses(losses)
fib.show_slice('end','Optimized end slice', show_issues=True)

## Interpolating and optimizing trajectories

In [ ]:
fib.interpolate_configuration(number_slices)
fib.show_3D_configuration('Interpolated configuration')

In [ ]:
fib.animate_slices('Interpolated configuration')

In [ ]:
losses = fib.optimize_configuration_heuristic(iters=100, repetitions=10)

In [ ]:
fib.show_losses(losses)
fib.show_3D_configuration('Optimized configuration')

In [ ]:
fib.animate_slices('Optimized configuration')

## Analyzing and adjusting the radii

In [ ]:
analysis = fib.get_full_analysis()
fib.show_3D_configuration_analysis(analysis, title='Location of configuration issues')
fib.show_analysis_distribution(analysis, title='Distribution of configuration issues')


In [ ]:
fixed_radii = fib.fix_radii()
fib.show_fixed_radii(fixed_radii)

In [ ]:
fib.set_radii(fixed_radii)
analysis = fib.get_full_analysis()
fib.show_3D_configuration_analysis(analysis, title='Location of configuration issues')
fib.show_analysis_distribution(analysis, title='Distribution of configuration issues')
        

In [ ]:
fib.set_radii(fixed_radii)
fib.animate_slices(title='Optimized configuration with fixed radii')

## Exporting (if desired)

In [ ]:
fib.save_result('out/fibers.txt')

In [ ]:
fib.save_mesh('out/fibers.obj', n=8, close_ends=True)

## Projecting and voxelizing

In [ ]:
thetas = 45
side = 250  # I use skimage iradon below, and it will only work correctly for odd side
projections = fib.project(thetas=thetas, bins=side, z=side)
fp.animate(projections)

In [ ]:
voxelization = fib.voxelize(xy=side, z=side, transition='smooth')
fp.animate(voxelization)

In [ ]:

import tifffile # pip install tifffile
tifffile.imwrite('out/projections.tiff', (projections/projections.max()).numpy())
tifffile.imwrite('out/voxelization.tiff', voxelization.numpy())


In [ ]:
fib.save_mesh('out/fibers_in_volume.obj', n=8, close_ends=False, xyz=(side, side))